In [1]:
# import subprocess
# subprocess.call(['sh', './train.sh'])

# Training

In [2]:
from itertools import chain
from s_clip_scripts.main import main
from s_clip_scripts.params import parse_args

fashion = False

if fashion:
    str_args = '''--train-data Fashion-ALL
            --label-ratio 0.1
            --val-data Fashion-ALL
            --keyword-path keywords/fashion/class-name.txt
            --epochs 10
            --method base  
    '''
else:
    str_args = ''' --train-data RS-ALL
            --label-ratio 0.2
            --val-data RS-ALL
            --imagenet-val RSICD-CLS \
            --keyword-path keywords/RS/class-name.txt
            --epochs 25
            --zeroshot-frequency 5  
            --method base
    '''
str_args = str_args.split('\n')
str_args = [s.strip() for s in str_args]
str_args = [s.split(' ') for s in str_args]
str_args = list(chain(*str_args))
str_args = [s for s in str_args if len(s) > 0]
args = parse_args(str_args)

In [3]:
%%time
checkpoint_path = main(args) # Calls the main.py function of S-CLIP

Making log path: ./checkpoint/2023_10_25-09_45_34-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_False-PL_method_None-vit_False
Running with a single process on device cuda:0.
Getting data...
RS-ALL
Split: val
Data size: 1094
Data size: 210
Data size: 58
RSICD-CLS
Split: val
['Airport', 'BareLand', 'BaseballField', 'Beach', 'Bridge', 'Center', 'Church', 'Commercial', 'DenseResidential', 'Desert', 'Farmland', 'Forest', 'Industrial', 'Meadow', 'MediumResidential', 'Mountain', 'Park', 'Parking', 'Playground', 'Pond', 'Port', 'RailwayStation', 'Resort', 'River', 'School', 'SparseResidential', 'Square', 'Stadium', 'StorageTanks', 'Viaduct', 'playfields']
RS-ALL
Split: train
Data size: 8734
Data size: 1680
Data size: 497
Dataset length: 10911
Labeled size: 2182
D1 indices: tensor([10892,  6980,  2351,  ...,  7321,  9452,  8881])
D2 indices: tensor([1016,  179, 5269,  ..., 3598, 8635, 3590])
Data got.
Start epoch 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.17 GiB total capacity; 5.53 GiB already allocated; 18.50 MiB free; 5.61 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Eval

In [4]:
%%time
checkpoint = checkpoint_path 
if 'Fashion' in checkpoint:
    zeroshot_datasets = ["Fashion200k-SUBCLS", "Fashion200k-CLS", "FashionGen-CLS", "FashionGen-SUBCLS", "Polyvore-CLS", ]
    retrieval_datasets = ["FashionGen", "Polyvore", "Fashion200k",]
else:
    zeroshot_datasets = ["RSICD-CLS", "UCM-CLS"] # "WHU-RS19", "RSSCN7", "AID" -> NOT WORKING bc of different data-loading workings
    retrieval_datasets = ["RSICD", "UCM", "Sydney"]

for dataset in zeroshot_datasets:
    str_args = ['--name', checkpoint, '--imagenet-val', dataset]
    args = parse_args(str_args)
    main(args)

for dataset in retrieval_datasets:
    str_args = ['--name', checkpoint, '--val-data', dataset]
    args = parse_args(str_args)
    main(args)

NameError: name 'checkpoint_path' is not defined